In [1]:
import os

2


In [6]:
%pwd

'/config/workspace'

In [5]:
os.chdir('../')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.MlProject.constants import *
from src.MlProject.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from MlProject import logger
from MlProject.utils.common import get_size

In [31]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! with following info: \n{headers}')
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [33]:
## pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-07 02:40:18,302: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-07 02:40:18,304: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-07 02:40:18,314: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-07 02:40:18,316: INFO: common: created directory at: artifacts]
[2024-01-07 02:40:18,317: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-07 02:40:18,714: INFO: 147164892: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "bbf554ab6635114f10b5c1f9b377f77ab0e269d66304b51eff0b352a6a245a15"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F0D8:77EA2:2C1A9:473D1:6599C12B
Accept-Ranges: bytes
Date: Sat, 06 J